<a href="https://colab.research.google.com/github/Fadhlin11/assignment9/blob/main/Collaborative_Filtering_Fadhlin_Sakina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!java --version
!python --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Python 3.9.16


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 11.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=2caf044ee710c55d325aecbc92c342b6642cf457f985469140fccec4900b77c0
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
# Check spark session
print(spark)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=60194ea802edeb8b99b9f04a2c93e0d832f84889efcd854506f99ba70975450c
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [ ]:
import wget

In [ ]:
file = wget.download("https://raw.githubusercontent.com/apache/spark/master/data/mllib/als/sample_movielens_ratings.txt")

In [ ]:
lines = spark.read.text("/content/sample_movielens_ratings.txt").rdd 
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

In [ ]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=10, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.192546456597573


In [ ]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{22, 3.8010364},...|
|    10|[{92, 3.057492}, ...|
|     0|[{92, 3.1616292},...|
|     1|[{62, 3.1842582},...|
|    21|[{52, 4.403841}, ...|
|    11|[{18, 4.849661}, ...|
|    12|[{46, 5.0798235},...|
|    22|[{75, 4.653639}, ...|
|     2|[{93, 4.6790524},...|
|    13|[{93, 3.294159}, ...|
|     3|[{51, 4.050485}, ...|
|    23|[{32, 5.032986}, ...|
|     4|[{52, 3.4730196},...|
|    24|[{52, 4.4299784},...|
|    14|[{52, 4.7036023},...|
|     5|[{55, 4.0875545},...|
|    15|[{46, 4.27542}, {...|
|    25|[{46, 3.279273}, ...|
|    26|[{22, 5.1169806},...|
|     6|[{25, 3.5653937},...|
+------+--------------------+
only showing top 20 rows



In [ ]:
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{17, 3.8273275},...|
|     40|[{2, 3.5446844}, ...|
|     10|[{17, 3.6742446},...|
|     50|[{23, 3.7607872},...|
|     80|[{3, 3.254015}, {...|
|     70|[{9, 2.1836958}, ...|
|     60|[{22, 2.7411869},...|
|     90|[{23, 4.673506}, ...|
|     30|[{11, 4.68598}, {...|
|      0|[{28, 2.710801}, ...|
|     31|[{8, 2.908982}, {...|
|     81|[{11, 3.0975916},...|
|     91|[{12, 2.8225703},...|
|      1|[{15, 2.995996}, ...|
|     41|[{28, 2.7816775},...|
|     61|[{6, 2.1431937}, ...|
|     51|[{26, 4.557882}, ...|
|     21|[{26, 2.6839693},...|
|     11|[{18, 3.3513012},...|
|     71|[{25, 3.1145916},...|
+-------+--------------------+
only showing top 20 rows



In [ ]:
# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{22, 5.1169806},...|
|    19|[{94, 3.4401348},...|
|    29|[{46, 4.1892414},...|
+------+--------------------+



In [ ]:
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     65|[{23, 4.292309}, ...|
|     26|[{15, 2.3206892},...|
|     29|[{8, 4.7745547}, ...|
+-------+--------------------+



Conclusion :

maxIter = 5 , regParam = 0.1 , 
RMSE = 1.0335768744191591

maxIter = 10 , regParam = 0.1 , 
RMSE = 0.9916857106326249

maxIter = 10 , regParam = 0.5 , 
RMSE = 1.192546456597573

The lower the RMSE, the better the model and its predictions. Therefore, maxIter = 10 , regParam = 0.1 is the best model.
